In [2]:
##《 Step 1 》
#skip duplicate 篩檔案完全重複的row
#All.txt格式和All-no-dup.txt都是『食材名, 單位, 重量, 大卡』
in_file = open('All.txt','r')
out_file = open('All-no-dup.txt','w')
unique = set()
for line in in_file.readlines():
    if line in unique: 
        continue 
    else:
        unique.add(line)
        out_file.write(line)
in_file.close()
out_file.close()

##《 Step 2》
#把第每一欄的東東切乾淨
in_file = open('All-no-dup.txt','r')
out_file = open('All-clean.txt','w')
for i in in_file.readlines(): #因為有些第一欄有多的逗號, 所以切四塊, 並精確滴寫回去
    a = i.split(',')[0] #食材名
    c = i.rsplit(',',3)[1] #單位
    d = i.rsplit(',',3)[2] #重量
    e = i.rsplit(',',3)[3] #大卡
    if '（' in a:
        b = a.split('（')[0]
        out_file.write((b+','+c+','+d+','+e))
    elif '(' in a:
        b = a.split('(')[0]
        out_file.write((b+','+c+','+d+','+e))
    else:
        out_file.write(i) 
        
in_file.close()
out_file.close()


In [3]:
##《 Step 3 》

#calculate kal per unit(g) 第五欄位每克有幾卡
#用正規表達法處理數字加其他字元
#可以先測試一下
#import re 
#Amm = re.match(r'(?P<num>\d+).*','100g')
#Bmm = re.match(r'(?P<num>\d+).*','100大卡')
#print Amm.group("num")
#print Bmm.group("num")
#結果 100,100 ; 可以成功切開第4欄惱人的g, 克, ml; 第5欄的卡,大卡...etc. 


import re 
in_file = open('All-clean.txt','r')
out_file = open('Calories_table1.txt','w')

for i in in_file.readlines():
    q1 = i.split(',')[0].strip() #食材名
    q2 = i.split(',')[1].strip() #單位
    q3 = i.split(',')[2].strip() #重量
    q4 = i.split(',')[3].strip() #
    Amm = re.match(r'(?P<num>\d+).*',q3) #eg. 100g; 100克 
    Bmm = re.match(r'(?P<num>\d+).*',q4) #eg. 100大卡; 100卡 
    if (Amm is not None) & (Bmm is not None):
        qq4 = Bmm.group("num") #eg. -> 100
        qq3 = Amm.group("num") #eg. -> 100
        q5 = float(qq4)/float(qq3) #計算大卡/重量(g)
        out_file.write(str(q1) +','+ str(qq3) +','+ str(qq4) +','+ str(q5) +','+ str(q2) +'\n') #重新排版一下; 數量單位放到最後
    else:
        out_file.write(str(q1) +','+ str(q3) +','+ str(q4) +',,'+ str(q2) +'\n')#重新排版一下; 數量單位放到最後
in_file.close()
out_file.close()

#進來滴All-clean.txt 的格式是「食材名, 單位, 重量, 大卡」
#出去滴Calories_table1.txt 的格式reshape成「食材名, 重量, 大卡, kal/g, 單位」


In [4]:
##《 Step 4 》

#找出同樣食材名稱with不同單位和熱量; = 第一欄一樣, 後面不一樣
#Calories_table1.txt 的格式「食材名, 重量, 大卡, kal/g, 單位」

in_file = open('Calories_table1.txt','r')
count = {} #築一個字典 {count[a]: value}
for i in in_file.readlines(): 
    a = i.split(',',4)[0] #食材名
    if a in count: #食材名有在字典嗎？
        count[a] = count[a] + 1 #有的話value就+1
    else:
        count[a] = 1 #沒有的話value就＝1
in_file.close()

#在count這個字典中，要是>1就是代表『第一欄一樣, 後面不一樣』; 拉出來append到dup list
dup=[] 
for k in count:
    #print k, count[k]
    if count[k] >1:
        dup.append(k)


In [5]:
##《 Step 5 》

#分別寫入兩個檔案 
#dups寫入ingre_dups.txt
in_file = open('Calories_table1.txt','r')
out_file1 = open('ingre_dups.txt','w')
QaaList= []
for Qaa in in_file.readlines(): #把Calories_table1.txt的每一行放到QaaList
    QaaList.append(Qaa)
for g in dup: #在dup字典裡面的食材
    for j in QaaList: 
        if g == j.split(',')[0]: #要是＝Calories_table1.txt的"第一欄"
            out_file1.write(j) #整行『j』寫入ingre_dups.txt
in_file.close()
out_file1.close()

In [6]:
#no dups寫入ingre_no_dups.txt
#類似上面寫法
in_file = open('Calories_table1.txt','r')
out_file1 = open('ingre_dups.txt','r')
out_file2 = open('ingre_no_dups.txt','w')
QaaList= []
QccList= []
for Qaa in in_file.readlines(): #把Calories_table1.txt的每一行放到QaaList
    QaaList.append(Qaa)
for Qcc in out_file1.readlines(): #把ingre_dups.txt的每一行放到QccList
    QccList.append(Qcc)

no_dups = []    
for Qxx in QaaList: 
    if Qxx not in QccList:
        no_dups.append(Qxx)
        out_file2.write(Qxx)
out_file2.close()
out_file1.close()
in_file.close()

In [ ]:
##《 Step 6 》
#use R to output ingre_dups1.txt

####問老歌，怎麼直接把單位合併在最後
#reshape?Aggregate? 

'''
Qoo = read.table("ingre_dups.txt",sep=",")
library(dplyr)
str(Qoo)

Qoo$V2 = as.integer(as.character(Qoo$V2))
Qoo$V3 = as.integer(as.character(Qoo$V3))

Qxx = 
  Qoo %>% 
  group_by(V1) %>% 
  summarise(WeightMean = mean(V2,na.rm = T),KalMean = mean(V3, na.rm = T),PerGramMean = mean(V4, na.rm = T))

write.table(Qxx, "ingre_dups1.txt",sep=",",quote = F, row.names = F,
              col.names = F)
Qyy =
select(Qoo, V1, V5) %>% 
  filter(V5 != '') %>% 
  distinct(V1,V5)

write.table(Qxx, "ingre_units.txt",sep=",",quote = F, row.names = F,
            col.names = F)

write.table(inner_join(Qxx, Qyy), "joined.txt",sep=",",quote = F, row.names = F,
            col.names = F)
'''


In [ ]:
##《 Step 7 》
#把食材褲同義詞建成字典格式

test_file1 = open('ingredient_lists.txt','r')
dic={}
lines_counter = 0
for i in test_file1.readlines():
    if test_file1 != '\n': #算一下file有幾行 = 有多少食材tag
        lines_counter += 1
    if '' in i.split(','): 
        a = i.split(',')[:i.split(',').index('')] #如果有空白的話，把後面空白截掉
        for j in a[1:]:
            dic[j] = a[0] #把同義詞對到食材
    else:
        a = i.split(',') 
        for j in a[1:]:
            dic[j] = a[0]
            
print '食材tag數量： ', lines_counter
QQQ = len(dic)-lines_counter
print '食材（含同義字）數量： ', QQQ

#for k in dic:
    #print k, dic[k]

In [8]:
##《 Step 8 》
#和現有食材字典dic match看看

#test_file1 = open('joined.txt','r') 
test_file1 = open('ingre_dups1.txt','r') #no dup version 先測試用 (單位要在append)
test_file2 = open('ingre_no_dups.txt','r') #no dups list
test_file4 = open('match.txt','w')

for i in test_file1.readlines():
    a = i.split(',')[0] #食材名
    if a in dic:
        #print dic[a] + ',' +i
        test_file4.write(dic[a] + ',' +i) #print 食材tag名 + 食材熱量細節
test_file1.close()


for i in test_file2.readlines():
    a = i.split(',')[0] #食材名
    if a in dic:
        #print dic[a] + ',' +i
        test_file4.write(dic[a] + ',' +i) #print 食材tag名 + 食材熱量細節
test_file4.close()
test_file2.close()

#出來的match.txt : tag名, 食材名, 重量, 大卡, kal/g, 單位

In [10]:
##《 Step 9 》
#算一下沒有match到的食材tag; 程式和##《 Step 5 》的概念一樣

test_file1 = open('ingredient_lists.txt','r')
test_file4 = open('match.txt','r')
test_file6 = open('unmatch_ingredients.txt','w')

QaaList= []
QccList= []
for j in test_file1.readlines():
    Qaa = j.split(',')[0]
    QaaList.append(Qaa)
for Qkk in test_file4.readlines():
    Qcc = Qkk.split(',')[0]
    QccList.append(Qcc)

cnt = []    
for Qxx in QaaList:    
    if Qxx not in QccList:
        cnt.append(Qxx)
        test_file6.write(Qxx+'\n')#還沒有熱量的主食材
print "沒有match到的食材tag: " + str(len(cnt))

test_file1.close()
test_file6.close()
test_file4.close()

沒有match到的食材tag: 294


In [12]:
##《 Step 10 》
#算出獨立食材名; 程式和##《 Step 4 》的概念一樣

qoo = open('match.txt','r')
aqoo = open('match-count.txt','w')
count = {}
for i in qoo.readlines():
    a = i.split(',')[1]
    if a in count:
        count[a] = count[a] + 1
    else:
        count[a] = 1
print '獨立食材 match ', len(count)
print '獨立食材 match%= ', format((float(len(count))/QQQ), '.2%')
for k in count:
    #print k,count[k]
    aqoo.write(k +' '+ str(count[k])+'\n')  
aqoo.close()
qoo.close()

獨立食材 match  1212
獨立食材 match%=  60.90%


In [13]:
##《 Step 11 》
#算出獨立tag名; 程式和##《 Step 4 》, ##《 Step 10 》的概念一樣

qoo = open('match.txt','r')
aqoo = open('match-count.txt','w')
count = {}
for i in qoo.readlines():
    a = i.split(',')[0]
    if a in count:
        count[a] = count[a] + 1
    else:
        count[a] = 1
print '食材tag match: ', len(count)
print '食材tag match%= ', format((float(len(count))/QQQ), '.2%')
for k in count:
    #print k,count[k]
    aqoo.write(k +' '+ str(count[k])+'\n')
    
aqoo.close()
qoo.close()


食材tag match:  772
食材tag match%=  38.79%
